# Finding Saddle-Point Solutions with Monomer Potential Fields

This script demonstrates how to find the saddle-point solution of AB diblock copolymer systems using monomer potential fields (such as $w_A$, $w_B$, and $w_C$). 

* This example is based on the equations in `PolymerFieldTheory/MonomerPotentialFields.ipynb`.

References:
* [(2017) Accelerating self-consistent field theory of block polymers in a variable unit cell](http://dx.doi.org/10.1063/1.4986643)

### 1. First-order iterative method

The potential fields are updated using the equations below:
\begin{align}
{\bf R}_w &= X{\boldsymbol\phi} − P{\bf w}, \\
w_i({\bf r}, \tau+1) &= w_i({\bf r}, \tau) + \lambda_i ({\bf R}_w)_i, \\
\end{align}
where $\lambda_i$ is an update rate and $\tau$ is the iteration number.

##### Example 1) AB-type polymeric systems 

\begin{align}
{\bf R}_w =
\left[\begin{array}{cc} 
0 & \chi N \\ 
\chi N & 0
\end{array}\right]
\left[\begin{array}{cc} 
\phi_A({\bf r}) \\ 
\phi_B({\bf r}) 
\end{array}\right] &−
\left[\begin{array}{cc} 
\ \ 1/2 & -1/2 \\ 
-1/2 & \ \ 1/2
\end{array}\right]
\left[\begin{array}{cc} 
w_A({\bf r}) \\ 
w_B({\bf r}) 
\end{array}\right],
\end{align}

\begin{align}
w_A({\bf r}, \tau+1) &= w_A({\bf r}, \tau) + \lambda_A \left[\chi N \phi_B({\bf r}) - \frac{1}{2}(w_A({\bf r},\tau)-w_B({\bf r},\tau)) \right], \\
w_B({\bf r}, \tau+1) &= w_B({\bf r}, \tau) + \lambda_B \left[\chi N \phi_A({\bf r}) + \frac{1}{2}(w_A({\bf r},\tau)-w_B({\bf r},\tau)) \right]. \\
\end{align}

### 2. Setting simulation parameters and initialization

| Parameter | Symbol | Value |
|:----------|:------:|------:|
| Reference chain length | $N_\text{Ref}$ | 100 |
| Contour step | $\Delta s$ | 1/100 |
| Reference length | $R_0$ | $b N_\text{Ref}^{1/2}$ |
| Box size X | $L_x$ | 2.0 $R_0$ |
| Box size Y | $L_y$ | 2.0 $R_0$ |
| Grid points X | $m_x$ | 64 |
| Grid points Y | $m_y$ | 64 |
| Segment length A | $b_A/b$ | 1.0 |
| Segment length B | $b_B/b$ | 1.0 |
| Interaction parameter | $\chi N$ | 20 |
| A-block fraction | $f$ | 0.5 |

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded OpenMP
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded FFT

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import polymerfts
from polymerfts import PropagatorSolver

# Simulation parameters
nx = [64,64]                       # grid number
lx = [2.0,2.0]                     # box size
stat_seg_lengths = {"A":1.0,       # statistical segment lengths
                    "B":1.0}        
ds = 0.01                          # contour step interval
chi_n = {"A,B":20}
monomer_types = ["A", "B"]  # monomer types

# AB diblock copolymer
blocks = [["A", 0.5, 0, 1],   # monomer type, contour length, start node, end node
          ["B", 0.5, 1, 2]]

# Create PropagatorSolver
solver = PropagatorSolver(
    nx=nx,
    lx=lx,
    ds=ds,
    bond_lengths=stat_seg_lengths,
    bc=["periodic", "periodic", "periodic", "periodic"],
    chain_model="continuous",
    method="pseudospectral",
    reduce_memory=False,
)

# Add AB diblock copolymer
solver.add_polymer(volume_fraction=1.0, blocks=blocks)

### 3. Initial potential fields

In [ ]:
w_A =  np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))   # sinusoidal potential field for A
w_B = -np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))   # sinusoidal potential field for B
w = {"A": np.reshape(w_A, np.prod(nx)), "B": np.reshape(w_B, np.prod(nx))}  # potential field dictionary

# Plot each image with the same vmin and vmax
vmin = min(w_A.min(), w_B.min())
vmax = max(w_A.max(), w_B.max())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Potential Fields")
im = axes[0].imshow(w_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)
im = axes[1].imshow(w_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)
axes[0].set(title='$w_A$', xlabel='y', ylabel='x')
axes[1].set(title='$w_B$', xlabel='y', ylabel='x')

fig.subplots_adjust(right=0.92)
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.8)
fig.show()

### 4. Finding saddle-point solutions

In [ ]:
tolerance = 1.0e-6  # convergence tolerance
lambda_update = 0.5  # fields update rate

# The number of components
M = len(monomer_types)

# X (χN matrix) and its inverse
matrix_chi = np.array([
    [0, chi_n["A,B"]],
    [chi_n["A,B"], 0]]
)
matrix_chi_inv = np.linalg.inv(matrix_chi)

# P matrix for field residuals
matrix_p = np.identity(M) - np.matmul(np.ones((M,M)), matrix_chi_inv)/np.sum(matrix_chi_inv)

def compute_concentrations(w):
    # For the given fields, compute the polymer statistics
    solver.compute_propagators(w)
    solver.compute_concentrations()

    # Compute total concentration for each monomer type
    phi = {}
    for monomer_type in monomer_types:
        phi[monomer_type] = solver.get_concentration(monomer_type)
    return phi

# Assign large initial value for the energy and error
energy_total = 1.0e20
error_level = 1.0e20

# Iteration begins here
print("---------- Run ----------")
print("iteration, mass error, total_partitions, free_energy, error_level")
for scft_iter in range(1000):
    # Compute total concentration for each monomer type
    phi = compute_concentrations(w)

    # Calculate the total energy
    energy_total = -solver._molecules.get_polymer(0).get_volume_fraction()/ \
                    solver._molecules.get_polymer(0).get_alpha() * \
                    np.log(solver.get_partition_function(0))
    energy_total -= np.mean(w["A"]*phi["A"]) + np.mean(w["B"]*phi["B"])
    energy_total += chi_n["A,B"]*np.mean(phi["A"]*phi["B"])

    # Calculate difference between current total density and target density
    phi_diff = phi["A"] + phi["B"] - 1.0

    # Calculate self-consistency error
    w_diff = np.zeros([M, solver.n_grid], dtype=np.float64) # array for output fields
    for i in range(M):
        for j in range(M):
            w_diff[i,:] += matrix_chi[i,j]*phi[monomer_types[j]] - matrix_p[i,j]*w[monomer_types[j]]

    # Keep the level of functional derivatives
    for i in range(M):
        w_diff[i] -= np.mean(w_diff[i])

    # error_level measures the "relative distance" between the input and output fields
    old_error_level = error_level
    error_level = 0.0
    error_normal = 1.0  # add 1.0 to prevent divergence
    for i in range(M):
        error_level += np.sum(w_diff[i]*w_diff[i])*np.prod(lx)/np.prod(nx)
        error_normal += np.sum(w[monomer_types[i]]*w[monomer_types[i]])*np.prod(lx)/np.prod(nx)
    error_level = np.sqrt(error_level/error_normal)

    # Print iteration # and error levels and check the mass conservation
    mass_error = np.mean(phi_diff)
    
    print("%8d %12.3E " % (scft_iter, mass_error), end=" [ ")
    for p in range(solver._molecules.get_n_polymer_types()):
        print("%13.7E " % (solver.get_partition_function(p)), end=" ")
    print("] %15.9f %15.7E " % (energy_total, error_level))

    # Conditions to end the iteration
    if error_level < tolerance:
        break

    # Update fields using simple gradient descent
    for i in range(M):
        w[monomer_types[i]] += lambda_update*w_diff[i]

### 5. Display the results

In [ ]:
# Get the ensemble average concentration for each monomer type
phi_A = np.reshape(solver.get_concentration("A"), nx)
phi_B = np.reshape(solver.get_concentration("B"), nx)

# Plot each image with the same vmin and vmax
vmin = min(phi_A.min(), phi_B.min())
vmax = max(phi_A.max(), phi_B.max())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Concentrations")
im = axes[0].imshow(phi_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r', vmin=vmin, vmax=vmax)
im = axes[1].imshow(phi_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0].set(title='$\phi_A$', xlabel='y', ylabel='x')
axes[1].set(title='$\phi_B$', xlabel='y', ylabel='x')

fig.subplots_adjust(right=0.92)
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.8)
fig.show()